# Task 7.1: Retrieve and Manage Experiment Results

This notebook provides a practical guide to the objects and methods used to retrieve and manage results from Qiskit Primitives jobs.

**Key Concepts Covered:**
- **Objective 1:** Understanding the `SamplerPubResult` object.
- **Objective 2:** Saving job results to disk and retrieving them.
- **Objective 3 & 4:** Exploring the attributes and methods of the `RuntimeJob` and `BasePrimitiveJob` classes.

All examples are designed to run locally using `AerSimulator`, with the cloud-specific sections clearly marked.

## Setup: Run a Simple Sampler Job

To explore the result objects, we first need to run a job. We'll create a simple Bell circuit and run it with the local `BackendSamplerV2`.

In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendSamplerV2 as Sampler
import numpy as np
import time
import json

# Helper function to create a Bell state circuit
def create_bell_circuit():
    """Create a Bell state (maximally entangled state) circuit."""
    qc = QuantumCircuit(2)
    qc.h(0)  # Apply Hadamard to qubit 0
    qc.cx(0, 1)  # Apply CNOT with control=0, target=1
    return qc

# 1. Create a circuit and backend
circuit = create_bell_circuit()
circuit.measure_all()
local_backend = AerSimulator()

# 2. Instantiate a local Sampler and run a job
sampler = Sampler(backend=local_backend)
job = sampler.run([(circuit,)], shots=1024)

# 3. Get the result object
# For local jobs, this is usually instantaneous.
result = job.result()
print("Successfully ran a local Sampler job.")
print(f"Job ID: {job.job_id()}")

Successfully ran a local Sampler job.
Job ID: adaf7d9d-e056-4939-ad63-d35f36f543c6


## Objective 1: Understanding `SamplerPubResult`

When you get the result from a V2 Primitives job, you get a `PrimitiveResult` object, which is a list of `PubResult` objects (one for each circuit, or "PUB," you submitted). Let's inspect the `PubResult` for our single job.

### Attributes: `data` and `metadata`

The `PubResult` has two main attributes:
*   `.data`: An object containing the core scientific output (e.g., bitstrings for a Sampler, EVs for an Estimator).
*   `.metadata`: A dictionary containing information about how the job was run (e.g., number of shots).

In [2]:
# Get the result for the first (and only) PUB
pub_result = result[0]

# Access the .data attribute
# For a Sampler, this is a DataBin object with a 'meas' field for measurements.
bitstring_data = pub_result.data.meas

# Access the .metadata attribute
metadata = pub_result.metadata

print(f"--- PubResult for Job '{job.job_id()}' ---")
print(f"\nMetadata: {metadata}")

# The data object has its own methods to access the results in different formats
print(f"\nFirst 5 Bitstrings: {bitstring_data.get_bitstrings()[:5]}")
print(f"\nCounts Dictionary: {bitstring_data.get_counts()}")

--- PubResult for Job 'adaf7d9d-e056-4939-ad63-d35f36f543c6' ---

Metadata: {'shots': 1024, 'circuit_metadata': {}}

First 5 Bitstrings: ['00', '00', '00', '00', '11']

Counts Dictionary: {'00': 510, '11': 514}


### Methods: `join_data()`

If you run multiple PUBs (multiple circuits) in a single job, you can use `join_data()` to combine their results.

In [3]:
# Run a job with two different circuits
circuit1 = create_bell_circuit()
circuit1.measure_all()

circuit2 = create_bell_circuit()
circuit2.x(0)  # Add X gate to flip the state
circuit2.measure_all()

job_multi = sampler.run([(circuit1,), (circuit2,)], shots=512)
result_multi = job_multi.result()

print("Results from first circuit:")
print(f"  Counts: {result_multi[0].data.meas.get_counts()}")

print("\nResults from second circuit:")
print(f"  Counts: {result_multi[1].data.meas.get_counts()}")

# Note: join_data() is used to combine DataBin objects with the same structure
# For demonstration, we can access individual PubResults and their data separately

Results from first circuit:
  Counts: {'00': 270, '11': 242}

Results from second circuit:
  Counts: {'10': 249, '01': 263}


## Objective 2: Save and Retrieve Jobs

For long-running jobs on real hardware, you won't want to wait in your notebook. You can submit a job, get its ID, and then retrieve the result later. This section also shows how to save results to and load them from your disk.

### Save Results to Disk

You can easily save your `PrimitiveResult` object to a JSON file for later analysis. This is useful for sharing results or for post-processing without having to run the job again.

In [4]:
# We need the Qiskit Runtime JSON encoder to handle the specific data types
from qiskit_ibm_runtime import RuntimeEncoder

# 'result' is the object we got from job.result() earlier
with open("sampler_result.json", "w") as file:
    json.dump(result, file, cls=RuntimeEncoder)

print("Result object successfully saved to 'sampler_result.json'")

Result object successfully saved to 'sampler_result.json'


### Load Results from Disk

In [5]:
# Use the Qiskit Runtime JSON decoder to correctly reconstruct the objects
from qiskit_ibm_runtime import RuntimeDecoder

with open("sampler_result.json", "r") as file:
    loaded_result = json.load(file, cls=RuntimeDecoder)

print("Result object successfully loaded from 'sampler_result.json'")
print(f"\nLoaded Result Object Type: {type(loaded_result)}")

# You can now access the data just as before
print(f"\nCounts from loaded data: {loaded_result[0].data.meas.get_counts()}")

Result object successfully loaded from 'sampler_result.json'

Loaded Result Object Type: <class 'qiskit.primitives.containers.primitive_result.PrimitiveResult'>

Counts from loaded data: {'00': 510, '11': 514}


### Retrieve Jobs from a Cloud Service (Optional)

**Note:** This section requires a configured `qiskit-ibm-runtime` account. If you don't have one, these cells will raise an `AccountNotFoundError`, which is expected. The `try...except` blocks will catch this and allow the notebook to continue.

In [6]:
import datetime

try:
    from qiskit_ibm_runtime import QiskitRuntimeService
    
    # This line will only work if you have an account saved locally
    service = QiskitRuntimeService()
    
    three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)
    jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
    
    print(f"Found {len(list(jobs_in_last_three_months))} jobs from the last 90 days.")
    if jobs_in_last_three_months:
        print("Showing the first three:")
        for i, job in enumerate(jobs_in_last_three_months[:3]):
            print(f"  {i+1}. Job {job.job_id()} - Status: {job.status()}")

except Exception as e:
    print(f"Cloud job retrieval skipped: {e}")
    print("This is expected if you don't have an IBM Quantum account configured.")

Cloud job retrieval skipped: "Unable to find account. Please make sure an account with the channel name 'ibm_quantum_platform' is saved."
This is expected if you don't have an IBM Quantum account configured.


### Retrieve a Specific Job by ID

If you know a job's ID, you can retrieve it directly. This is useful when you submit a job, save its ID, and come back later to get the results.

In [ ]:
try:
    from qiskit_ibm_runtime import QiskitRuntimeService
    
    service = QiskitRuntimeService()
    
    # Get the most recent successful job for demonstration
    successful_jobs = [j for j in service.jobs(limit=100) if j.status() == "DONE"]
    
    if successful_jobs:
        successful_job = successful_jobs[0]
        job_id = successful_job.job_id()
        print(f"Found successful job: {job_id}")
        
        # Retrieve the job by its ID
        retrieved_job = service.job(job_id)
        retrieved_result = retrieved_job.result()
        
        print(f"Successfully retrieved job {job_id}")
        print(f"Result: {retrieved_result}")
    else:
        print("No successful jobs found in recent history.")
        
except Exception as e:
    print(f"Job retrieval skipped: {e}")
    print("This is expected if you don't have an IBM Quantum account configured.")

Job retrieval skipped: "Unable to find account. Please make sure an account with the channel name 'ibm_quantum_platform' is saved."
This is expected if you don't have an IBM Quantum account configured.


## Objective 3 & 4: The `RuntimeJob` and `BasePrimitiveJob`

When you run a job, you get back a `Job` object. For local `BackendPrimitives`, this is a `PrimitiveJob`. For the runtime service, it's a `RuntimeJob`. Both inherit from `BasePrimitiveJob` and share a common set of methods for managing and querying the job's state.

**Important Note:** Some methods are only available on `RuntimeJob` (cloud jobs), not on `PrimitiveJob` (local jobs). We'll clearly mark which is which.

### Job Management Example

Let's run a job on the local simulator and use the job object's methods to monitor its status. For a local job, this will happen very quickly, but on real hardware, a job can stay in the `QUEUED` and `RUNNING` states for a while.

In [8]:
from qiskit.circuit.library import EfficientSU2
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.providers.jobstatus import JobStatus

# Create a slightly more complex circuit to give the simulator a moment of work
circuit = EfficientSU2(10, reps=4, entanglement='linear')
circuit.measure_all()
params = np.random.rand(circuit.num_parameters)

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=local_backend)
isa_circuit = pm.run(circuit)

# Submit the job
job = sampler.run([(isa_circuit, params)])

print(f"Submitted job with ID: {job.job_id()}")

# --- Polling for Job Status ---
# This loop mimics how you would check on a long-running cloud job.
max_checks = 50
checks = 0
while not job.in_final_state() and checks < max_checks:
    # JobStatus is an Enum with values like QUEUED, RUNNING, DONE, ERROR, CANCELLED
    status = job.status()
    print(f"Job status is currently: {status}")
    time.sleep(0.1)  # Wait a moment before checking again
    checks += 1

final_status = job.status()
print(f"\nJob finished with final status: {final_status}")

# --- Accessing Results ---
if job.done():
    result = job.result()
    print("Job completed successfully!")
    print(f"Sample counts: {result[0].data.meas.get_counts()}")
else:
    # For local jobs, errors are raised immediately when calling result()
    # Cloud jobs have additional methods like errored() and error_message()
    status = job.status()
    print(f"Job is in state: {status}")

Submitted job with ID: 9ada500c-a2b4-4b9e-bf19-a649cd083e50
Job status is currently: JobStatus.RUNNING

Job finished with final status: JobStatus.DONE
Job completed successfully!
Sample counts: {'1100110011': 7, '1110000100': 6, '0101110101': 1, '1110100101': 2, '1101100000': 11, '1011110000': 7, '1000000010': 6, '0111000000': 3, '1100111000': 5, '0001000000': 12, '0001001000': 3, '1000100001': 3, '1011111100': 2, '0101100111': 1, '0110001000': 5, '0111010000': 24, '0001110000': 5, '1010101101': 2, '1000000000': 21, '1101111000': 4, '1100110000': 8, '0111011110': 5, '0100001000': 2, '0000111011': 1, '0010111000': 2, '0011001100': 2, '1101010110': 1, '0101101101': 3, '1100100000': 5, '0000000000': 25, '1000010000': 4, '1110100000': 6, '1011001000': 1, '0000100000': 3, '0000000010': 8, '0011000110': 4, '0111111000': 8, '1111101100': 1, '1011010010': 3, '1101100101': 2, '1111011110': 3, '1010110001': 3, '0100100100': 2, '0001011100': 5, '0111100000': 12, '0111101000': 1, '1001100110': 1, 

/var/folders/8m/0wg1hssn6n79tjc8mh6p_spr0000gn/T/ipykernel_43803/51717147.py:6: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  circuit = EfficientSU2(10, reps=4, entanglement='linear')


### Key Job Attributes and Methods

The `job` object has many useful methods and attributes. Here are the most important ones:

**Methods Available on Both PrimitiveJob (local) and RuntimeJob (cloud):**
* `job.status()`: Returns the job status (e.g., `JobStatus.RUNNING`).
* `job.in_final_state()`: Returns `True` if the job is done, errored, or cancelled.
* `job.done()`: Returns `True` only if the job finished successfully.
* `job.running()`: Returns `True` if the job is currently executing.
* `job.cancelled()`: Returns `True` if the job was cancelled.
* `job.result()`: **Blocks execution** until the job is in a final state and then returns the `PrimitiveResult` object.
* `job.job_id()`: Returns the unique string ID for the job.
* `job.cancel()`: Attempts to cancel a queued or running job.

**Methods ONLY Available on RuntimeJob (cloud jobs):**
* `job.errored()`: Returns `True` if the job failed (not available on local PrimitiveJob).
* `job.error_message()`: Returns the error message if the job failed.
* `job.backend()`: Returns the backend object (may not work on all job types).
* `job.queue_info()`: Information about queue position.
* `job.queue_position()`: The job's position in the queue.
* `job.logs()`: Retrieve job logs.
* `job.metrics()`: Performance metrics for the job.
* `job.update_tags()`: Update job tags.

**RuntimeJob-Specific Attributes:**
* `job.creation_date`
* `job.tags`
* `job.session_id`
* `job.usage_estimation`

In [9]:
# --- Demonstration of job methods that work on ALL job types ---

# We can get the job ID
print(f"Job ID: {job.job_id()}")

# Check if the job is in a final state (should be True now)
print(f"Is job in a final state? {job.in_final_state()}")

# Check specific status flags (these work on all jobs)
print(f"Is job done? {job.done()}")
print(f"Is job running? {job.running()}")
print(f"Is job cancelled? {job.cancelled()}")

# Note: errored() is only available on RuntimeJob
# For local jobs, check status directly:
print(f"Job status: {job.status()}")

Job ID: 9ada500c-a2b4-4b9e-bf19-a649cd083e50
Is job in a final state? True
Is job done? True
Is job running? False
Is job cancelled? False
Job status: JobStatus.DONE


## Summary

In this notebook, we covered:

1. **PubResult Structure**: The `data` and `metadata` attributes that contain your results
2. **Saving/Loading Results**: Using `RuntimeEncoder` and `RuntimeDecoder` to persist results to disk
3. **Job Retrieval**: How to retrieve jobs from IBM Quantum (cloud-specific)
4. **Job Management**: Understanding the difference between PrimitiveJob (local) and RuntimeJob (cloud)
   - Common methods that work on both
   - Cloud-specific methods for monitoring and error handling

These tools are essential for managing long-running quantum experiments, especially when working with real quantum hardware where jobs may be queued for extended periods.

## Practice Questions

**1. In a V2 `PubResult` object, where is the core scientific data (such as bitstrings or expectation values) stored?**

A) In the `.metadata` attribute

B) In the `.data` attribute

C) In the `.results` attribute

D) In the `.value` attribute

***Answer:***
<Details>
<br/>
B) In the `.data` attribute
</Details>

---

**2. Which method should you use to reliably check if a job has completed successfully?**

A) `job.in_final_state()`

B) `job.running()`

C) `job.done()`

D) `job.status() == 'COMPLETED'`

***Answer:***
<Details>
<br/>
C) `job.done()` (Returns True only if the job finished successfully, excluding cancellations or errors)
</Details>

---

**3. Which of the following methods is typically available on a cloud-based `RuntimeJob` but not on a local `PrimitiveJob`?**

A) `job.status()`

B) `job.job_id()`

C) `job.error_message()`

D) `job.result()`

***Answer:***
<Details>
<br/>
C) `job.error_message()`
</Details>